In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline  
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold


from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import StackingClassifier

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score

import warnings
warnings.filterwarnings("ignore")


In [4]:
df = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Cases\Satellite Imaging\Satellite.csv', sep=';')
df.head()

,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,...,x.28,x.29,x.30,x.31,x.32,x.33,x.34,x.35,x.36,classes
0,92,115,120,94,84,102,106,79,84,102,...,104,88,121,128,100,84,107,113,87,grey soil
1,84,102,106,79,84,102,102,83,80,102,...,100,84,107,113,87,84,99,104,79,grey soil
2,84,102,102,83,80,102,102,79,84,94,...,87,84,99,104,79,84,99,104,79,grey soil
3,80,102,102,79,84,94,102,79,80,94,...,79,84,99,104,79,84,103,104,79,grey soil
4,84,94,102,79,80,94,98,76,80,102,...,79,84,103,104,79,79,107,109,87,grey soil


In [5]:
X = df.drop(['classes'],axis=1)
y = df['classes']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [7]:
nb = GaussianNB()
knn = KNeighborsClassifier(n_neighbors=5) 
dtc = DecisionTreeClassifier(random_state=24)  
svm = SVC(probability=True, random_state=24)
lr = LogisticRegression(random_state=24)
rf = RandomForestClassifier(random_state=24)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

stack = StackingClassifier(
    estimators=[
        ('knn', knn),
        ('dtc', dtc),
        ('nb', nb),    
        ('svm', svm), 
    ],
)
params = {
    # Fror linear regression as final estimator
    'final_estimator': [lr, rf],
    'passthrough': [True, False]
    } 


grid = GridSearchCV(
    estimator=stack, 
    param_grid=params,
    cv=kfold, 
    scoring='neg_log_loss'
)


In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_score_, grid.best_params_

In [ ]:
best_model = grid.best_estimator_

import pickle
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)


with open('best_model.pkl', 'rb') as f:
    best_model = pickle.load(f)

y_pred = best_model.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
tst_df = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Cases\Satellite Imaging\tst_satellite.csv')